In [1]:
%load_ext autoreload
%autoreload 2

# Data Exploration

In [ ]:
import polars as pl

from roar import ALL_VEHICLES, DATA_DIR, TYRE_CLEAN_NAMES, VEHICLE_CLEAN_NAMES

In [3]:
# Some data cleaning for better usability
all_channel_names = pl.read_csv(DATA_DIR / "Extras" / "all_measurement_channels name.csv")
all_channel_names = all_channel_names.with_columns(
    pl.col("Fahrzeug")
    .str.replace(" ", "")
    .str.split(",")
    .replace(VEHICLE_CLEAN_NAMES)
    .replace({tuple(["alle"]): ALL_VEHICLES}),
    pl.col("Relevanz").cast(pl.Categorical),
)
all_channel_names

Beschreibung,Kanalname,Synonym 1,Synonym 2,Fahrzeug,Relevanz
str,str,str,str,list[str],cat
"""Fzg-CAN - Antrieb""","""Antrieb""",null,null,"[""ID.4""]","""nein"""
"""Fzg-CAN""","""BCM1_Aussen_Temp""",null,null,"[""eGolf"", ""ID.4"", … ""Taycan""]","""ja"""
"""HEAD-CAN - labCTRL""","""CAN1_labCTRLI_""",null,null,"[""eGolf"", ""ID.4"", … ""Taycan""]","""nein"""
"""HEAD-CAN - labDX""","""CAN1_labDX_B""",null,null,"[""eGolf"", ""ID.4"", … ""Taycan""]","""nein"""
"""SIS - Trailing Edge - K1""","""Ch_1_labV12""","""TrailK1""",null,"[""eGolf"", ""ID.4"", … ""Taycan""]","""ja"""
…,…,…,…,…,…
"""iMAR""","""v_east_CAN_Sig_""",null,null,"[""eGolf"", ""ID.4"", … ""Taycan""]","""ja"""
"""iMAR""","""v_north_CAN_Sig""",null,null,"[""eGolf"", ""ID.4"", … ""Taycan""]","""ja"""
"""iMAR""","""v_up_CAN_Sig_11""","""v_up_CAN_Sig_21""",null,"[""eGolf"", ""ID.4"", … ""Taycan""]","""ja"""


In [4]:
messungen = pl.read_csv(DATA_DIR / "Extras" / "messungen.csv")
messungen

Messtyp,Geschwindigkeit min (km/h),Geschwindigkeit max (km/h),Beschleunigung (m/s2),Abbrev
str,i64,i64,i64,str
"""v1""",45,80,null,"""meas0"""
"""v2""",80,100,null,"""meas1"""
"""v3""",100,120,null,"""meas2"""
"""a1""",null,null,1,"""meas3"""
"""a2""",null,null,2,"""meas4"""
"""a3""",null,null,3,"""meas5"""


In [5]:
# Does not have a lot of relevance right now -> just load it
strecken = pl.read_csv(DATA_DIR / "Extras" / "strecken.csv")
strecken

ID,Name,GPS,GPS Start,GPS Ende
i64,str,str,str,str
150,"""Unknown Test Track""",null,null,null
211,"""ika Teststrecke""","""50.79101691007796, 6.048780870…","""50.79115036301304, 6.049006938…","""50.790885556264705, 6.04855011…"
259,"""ika - ISO""","""50.79101691007796, 6.048780870…","""50.79115036301304, 6.049006938…","""50.790885556264705, 6.04855011…"


In [6]:
tyre_road_information = pl.read_csv(DATA_DIR / "Extras" / "tyre_road_information.csv")
tyre_road_information = tyre_road_information.with_columns(
    pl.col("Fahrzeug").replace(VEHICLE_CLEAN_NAMES)
)
tyre_road_information

Reifen Nummer,Hersteller,Name,Dimension,Fahrzeug
i64,str,str,str,str
1,"""Continental""","""EcoContact 6 Q ""","""235/55R19 105H""","""ID.4"""
3,"""Uniroyal""","""RainSport 5 ""","""235/55R19 105V""","""ID.4"""
6,"""Continental""","""Premium Contact 6 AO""","""255/50R20 109H""","""Q8"""
10,"""Pirelli""","""P-Zero R""","""275/35R21 103Y""","""Taycan"""
12,"""Hankook""","""Ventus S1 evo3 ev""","""255/50R20 109H""","""Q8"""
13,"""BFGoodrich""","""Summer SRTT""","""225/45R17 94V""","""eGolf"""


In [ ]:
vehicle_and_tyre_features = pl.read_csv(DATA_DIR / "Extras" / "vehicle_and_tyre_feature.csv")
vehicle_and_tyre_features = vehicle_and_tyre_features.with_columns(
    pl.col("ReifenID").str.replace_all(r"\D", "").str.to_integer(),
    pl.col("Fahrzeug").replace(VEHICLE_CLEAN_NAMES),
    pl.col("Reifenmodell").replace(TYRE_CLEAN_NAMES),
)
vehicle_and_tyre_features

Fahrzeug,ReifenID,Reifenmodell,,_duplicated_0,_duplicated_1,_duplicated_2,_duplicated_3,_duplicated_4,_duplicated_5,_duplicated_6,_duplicated_7,_duplicated_8,_duplicated_9,_duplicated_10,_duplicated_11,_duplicated_12,_duplicated_13,_duplicated_14,_duplicated_15,_duplicated_16,_duplicated_17
str,i64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
"""Q8""",6,"""PremiumContact 6 AO""",2.5,11.9,2324.0,739.8,71.0,6.9,278.15,27.5,6.89,177.0,38.0,38.9,9.56,315.15,68.0,201.5,7.0,14.1,null
"""Q8""",12,"""Ventus S1 evo 3 ev""",2.5,15.3,2426.0,768.3,72.0,6.3,312.2,31.0,6.98,201.0,51.5,39.2,9.2,312.2,72.0,201.0,6.3,15.3,null
"""ID.4""",1,"""EcoContact 6 Q""",2.5,11.0,2349.5,747.9,64.5,6.0,262.45,31.9,4.51,173.5,34.5,27.5,6.89,262.45,64.5,173.5,6.0,11.0,null
"""ID.4""",3,"""RainSport 5""",2.5,11.7,2330.0,741.7,64.5,7.9,258.85,29.8,7.28,173.0,76.5,33.8,7.36,258.85,64.5,173.0,7.9,11.7,null
"""Taycan""",10,"""P-Zero """,2.4,13.5,2283.5,285.3,69.5,null,320.9,39.2,9.2,207.0,31.5,0.0,0.0,320.9,69.5,207.0,0.0,13.5,null
"""eGolf""",13,"""Summer SRTT """,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null


## h5 files

In [25]:
# List all h5 files in the data directory
h5_files = list((DATA_DIR).rglob("*.h5"))
files_df = pl.DataFrame(
    {
        "file_path": h5_files,
        "stem": [f.stem for f in h5_files],
        "vehicle": [file.parent.parent.parent.name for file in h5_files],
        "tyre": [file.parent.name for file in h5_files],
        "track": [file.parent.parent.name for file in h5_files],
        "conventional_naming": [f.stem.startswith("track") for f in h5_files],
    }
)
files_df

file_path,stem,vehicle,tyre,track,conventional_naming
object,str,str,str,str,bool
/Users/moritzfeik/Developer/ROAR/data/ID.4/track150/tyre1/track150_ID.4_tyre1_meas3_2p5_1_2025-08-07_11-28-54.h5,"""track150_ID.4_tyre1_meas3_2p5_…","""ID.4""","""tyre1""","""track150""",true
/Users/moritzfeik/Developer/ROAR/data/ID.4/track150/tyre1/track150_ID.4_tyre1_meas1_2p5_1_2025-08-07_11-23-14.h5,"""track150_ID.4_tyre1_meas1_2p5_…","""ID.4""","""tyre1""","""track150""",true
/Users/moritzfeik/Developer/ROAR/data/ID.4/track150/tyre1/track150_ID.4_tyre1_meas2_2p5_1_2025-08-07_11-25-10.h5,"""track150_ID.4_tyre1_meas2_2p5_…","""ID.4""","""tyre1""","""track150""",true
/Users/moritzfeik/Developer/ROAR/data/ID.4/track150/tyre1/track150_ID.4_tyre1_meas3_2p5_1_2025-08-07_11-27-11.h5,"""track150_ID.4_tyre1_meas3_2p5_…","""ID.4""","""tyre1""","""track150""",true
/Users/moritzfeik/Developer/ROAR/data/ID.4/track150/tyre1/track150_ID.4_tyre1_meas1_2p5_1_2025-08-07_11-22-29.h5,"""track150_ID.4_tyre1_meas1_2p5_…","""ID.4""","""tyre1""","""track150""",true
…,…,…,…,…,…
/Users/moritzfeik/Developer/ROAR/data/Taycan/track211/tyre10/track211_Taycan_tyre10_meas2_2p5_1_2025-09-23_17-05-11.h5,"""track211_Taycan_tyre10_meas2_2…","""Taycan""","""tyre10""","""track211""",true
/Users/moritzfeik/Developer/ROAR/data/Taycan/track211/tyre10/track211_Taycan_tyre10_meas1_2p5_1_2025-09-23_17-00-35.h5,"""track211_Taycan_tyre10_meas1_2…","""Taycan""","""tyre10""","""track211""",true
/Users/moritzfeik/Developer/ROAR/data/Taycan/track211/tyre10/track211_Taycan_tyre10_meas3_2p5_1_2025-09-23_17-08-15.h5,"""track211_Taycan_tyre10_meas3_2…","""Taycan""","""tyre10""","""track211""",true


In [ ]:
# Extract measurement and date
files_df = files_df.with_columns(
    pl.when(pl.col("conventional_naming"))
    .then(pl.col("stem").str.extract(r"(meas\d+)", 1))
    .otherwise(
        pl.when(pl.col("stem").str.starts_with("b"))
        .then(
            pl.concat_str(
                [
                    pl.col("stem").str.extract(r"^(b\d+)", 1),
                    pl.lit("_vr"),
                    pl.col("stem").str.extract(r"vr(\d+)", 1),
                ]
            )
        )
        .otherwise(
            pl.concat_str(
                [
                    pl.lit("vr"),
                    pl.col("stem").str.extract(r"vr(\d+)", 1),
                ]
            )
        )
    )
    .alias("masurement"),
    # extract timestamp (as string)
    pl.col("stem")
    .str.extract(r"(\d{4}-\d{2}-\d{2}_\d{2}-\d{2}-\d{2})", 1)
    .str.strptime(pl.Datetime, "%Y-%m-%d_%H-%M-%S")
    .alias("date"),
)
files_df.head(10)

file_path,stem,vehicle,tyre,track,conventional_naming,masurement,date
object,str,str,str,str,bool,str,datetime[μs]
/Users/moritzfeik/Developer/ROAR/data/ID.4/track150/tyre1/track150_ID.4_tyre1_meas3_2p5_1_2025-08-07_11-28-54.h5,"""track150_ID.4_tyre1_meas3_2p5_…","""ID.4""","""tyre1""","""track150""",true,"""meas3""",2025-08-07 11:28:54
/Users/moritzfeik/Developer/ROAR/data/ID.4/track150/tyre1/track150_ID.4_tyre1_meas1_2p5_1_2025-08-07_11-23-14.h5,"""track150_ID.4_tyre1_meas1_2p5_…","""ID.4""","""tyre1""","""track150""",true,"""meas1""",2025-08-07 11:23:14
/Users/moritzfeik/Developer/ROAR/data/ID.4/track150/tyre1/track150_ID.4_tyre1_meas2_2p5_1_2025-08-07_11-25-10.h5,"""track150_ID.4_tyre1_meas2_2p5_…","""ID.4""","""tyre1""","""track150""",true,"""meas2""",2025-08-07 11:25:10
/Users/moritzfeik/Developer/ROAR/data/ID.4/track150/tyre1/track150_ID.4_tyre1_meas3_2p5_1_2025-08-07_11-27-11.h5,"""track150_ID.4_tyre1_meas3_2p5_…","""ID.4""","""tyre1""","""track150""",true,"""meas3""",2025-08-07 11:27:11
/Users/moritzfeik/Developer/ROAR/data/ID.4/track150/tyre1/track150_ID.4_tyre1_meas1_2p5_1_2025-08-07_11-22-29.h5,"""track150_ID.4_tyre1_meas1_2p5_…","""ID.4""","""tyre1""","""track150""",true,"""meas1""",2025-08-07 11:22:29
/Users/moritzfeik/Developer/ROAR/data/ID.4/track150/tyre1/track150_ID.4_tyre1_meas1_2p5_1_2025-08-07_11-21-28.h5,"""track150_ID.4_tyre1_meas1_2p5_…","""ID.4""","""tyre1""","""track150""",true,"""meas1""",2025-08-07 11:21:28
/Users/moritzfeik/Developer/ROAR/data/ID.4/track150/tyre1/track150_ID.4_tyre1_meas2_2p5_1_2025-08-07_11-26-06.h5,"""track150_ID.4_tyre1_meas2_2p5_…","""ID.4""","""tyre1""","""track150""",true,"""meas2""",2025-08-07 11:26:06
/Users/moritzfeik/Developer/ROAR/data/ID.4/track150/tyre1/track150_ID.4_tyre1_meas3_2p5_1_2025-08-07_11-28-02.h5,"""track150_ID.4_tyre1_meas3_2p5_…","""ID.4""","""tyre1""","""track150""",true,"""meas3""",2025-08-07 11:28:02
/Users/moritzfeik/Developer/ROAR/data/ID.4/track150/tyre1/track150_ID.4_tyre1_meas2_2p5_1_2025-08-07_11-24-13.h5,"""track150_ID.4_tyre1_meas2_2p5_…","""ID.4""","""tyre1""","""track150""",true,"""meas2""",2025-08-07 11:24:13
